In [40]:
# 匯入必要套件
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [41]:
# 資料讀取
x_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
x_test = pd.read_csv("X_test.csv")

df_train = x_train.merge(y_train, on="Index")
X = df_train.drop(columns=["Index", "Price"])
y = df_train["Price"]
X_test_submit = x_test.drop(columns=["Index"])
test_index = x_test["Index"]

In [42]:
# 特徵工程函式（結合房屋結構 + 建年修正 + 高低價模式）
def feature_engineering(df):
    df = df.copy()

    # 房屋總間數（房 + 廳 + 衛）
    df["total_rooms"] = df["建物現況格局-房"] + df["建物現況格局-廳"] + df["建物現況格局-衛"]

    # 主建物面積佔比
    df["main_building_ratio"] = df["主建物面積"] / (df["建物移轉總面積平方公尺"] + 1e-3)

    # 車位單價
    df["parking_unit_price"] = df["車位總價元"] / (df["車位移轉總面積(平方公尺)"] + 1e-3)

    # 總可使用空間
    df["total_usable_area"] = df["主建物面積"] + df["附屬建物面積"] + df["陽台面積"]

    # ✅ 建年修正（民國 → 西元），並計算屋齡
    df["built_year"] = df["built_year"].fillna(df["built_year"].median())
    df["built_year_AD"] = df["built_year"] + 1911
    df["building_age"] = 2023 - df["built_year_AD"]

    # 距離台北車站（25.0478, 121.5171）的距離（歐幾里得距離）
    df["distance_to_station"] = ((df["Latitude"] - 25.0478)**2 + (df["Longitude"] - 121.5171)**2) ** 0.5

    # ✅ 高價房模式特徵
    df["is_large_house"] = (df["建物移轉總面積平方公尺"] > 100).astype(int)
    df["has_elevator"] = df["電梯_有"]
    df["room_density"] = df["total_rooms"] / (df["建物移轉總面積平方公尺"] + 1e-3)
    df["has_parking"] = (df["車位移轉總面積(平方公尺)"] > 0).astype(int)
    df["usable_area_ratio"] = df["total_usable_area"] / (df["建物移轉總面積平方公尺"] + 1e-3)

    # ✅ 低價房特徵模式
    df["is_unusual_built_year"] = (df["built_year"] == 0) | (df["built_year"] > 110)
    df["has_zero_rooms"] = (df["total_rooms"] == 0).astype(int)
    df["tiny_main_area"] = (df["主建物面積"] < 40).astype(int)

    # ✅ 高價區標記（來自 top30 統計）
    core_areas = ["鄉鎮市區_內湖區", "鄉鎮市區_大安區", "鄉鎮市區_中正區",
                  "鄉鎮市區_新店區", "鄉鎮市區_汐止區", "鄉鎮市區_新莊區", "鄉鎮市區_文山區"]
    df["is_core_town"] = df[core_areas].sum(axis=1).clip(upper=1)

    # ✅ 新增樓層邏輯特徵
    df["is_low_rise"] = (df["total_floor"] <= 5).astype(int)
    df["is_high_without_elevator"] = ((df["total_floor"] >= 7) & (df["電梯_無"] == 1)).astype(int)
    df["is_fourth_floor"] = (df["sell_floor"] == 4).astype(int)
    df["is_top_floor"] = (df["sell_floor"] == df["total_floor"]).astype(int)
    df["floor_ratio"] = df["sell_floor"] / (df["total_floor"] + 1e-3)

    return df


In [43]:
# 應用特徵工程
X = feature_engineering(X)
X_test_submit = feature_engineering(X_test_submit)

# 特徵標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test_submit)

# 將 scaled array 轉回 dataframe 以利 XGBoost 運作
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)

In [44]:
# 建立 XGBoost 模型
xgb_model = xgb.XGBRegressor(
    n_estimators=10000,
    max_depth=8,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.6,
    random_state=42
)

In [45]:
# 模型訓練與驗證
xgb_model.fit(X_scaled_df, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.02, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=10000,
             n_jobs=None, num_parallel_tree=None, ...)

In [46]:
# 產生測試資料預測結果
final_preds = xgb_model.predict(X_test_scaled_df)

# 建立提交檔案
submission = pd.DataFrame({
    "Index": test_index,
    "Price": final_preds
})
submission.to_csv("r13725052_周哲群_submission.csv", index=False)
print("Submission file saved.")


Submission file saved.
